In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import cupy as cp
from sklearnex import patch_sklearn


In [2]:
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

def mainBareBones():
    print("Loading dataset...")
    try:
        # Load main dataset
        df = pd.read_csv('Active_Wiretap_dataset.csv', 
                         header=None,
                         low_memory=False)
        
        # Load labels and handle mixed types
        labels = pd.read_csv('Active_Wiretap_labels.csv', 
                            names=['label'], 
                            header=None,
                            low_memory=False)
        
        # Ensure datasets have the same number of rows
        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        
        # Convert labels to numeric
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')
        
        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return

    # Step 2: Preprocess the data
    print("\nPreprocessing data...")
    n_features = df.shape[1]
    column_names = [f"feature_{i}" for i in range(n_features)]
    df.columns = column_names
    df['label'] = labels['label']

    # Step 3: Optimize memory usage
    print("Optimizing memory usage...")
    df = optimize_dtypes(df)

    # Step 4: Clean data
    print("Cleaning data...")
    # Remove any rows with missing values
    df = df.dropna()
    # Remove any rows where the label is NaN
    df = df[df['label'].notna()]

    print(f"Shape after cleaning: {df.shape}")

    # Step 5: Split features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    # Step 6: Split the data
    print("\nSplitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Step 7: Feature selection
    print("Performing feature selection...")
    selector = SelectKBest(score_func=f_classif, k=min(30, X.shape[1]))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Step 8: Train model
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train_selected, y_train)
    print("Model training completed!")

    # Step 9: Predictions
    print("Making predictions...")
    y_pred = model.predict(X_test_selected)

    # Step 10: Evaluation
    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Additional information
    print("\nProcess completed!")

    print("\nDataset Information:")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{y.value_counts()}")

if __name__ == "__main__":
    mainBareBones()

Loading dataset...
Dataset shape: (2278689, 115)
Labels shape: (2278689, 1)
Label values found: [nan  0.  1.]

Preprocessing data...
Optimizing memory usage...
Cleaning data...
Shape after cleaning: (2278688, 116)

Splitting data into train and test sets...
Performing feature selection...

Training the MLP model...
Model training completed!
Making predictions...

Model Evaluation:

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    406644
         1.0       0.94      0.98      0.96    276963

    accuracy                           0.97    683607
   macro avg       0.96      0.97      0.97    683607
weighted avg       0.97      0.97      0.97    683607


Confusion Matrix:
[[389456  17188]
 [  4378 272585]]

Process completed!

Dataset Information:
Total samples: 2278688
Number of features: 115
Number of selected features: 30
Label distribution:
label
0.0    1355473
1.0     923215
Name: count, dtype: int64


In [3]:
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = cp.array(df[col], dtype=cp.float32).get()
        elif df[col].dtype == 'int64':
            df[col] = cp.array(df[col], dtype=cp.int32).get()
    return df

def mainCuPY():
    print("Loading dataset...")
    try:
        # Load main dataset
        df = pd.read_csv('Active_Wiretap_dataset.csv', 
                         header=None,
                         low_memory=False)
        
        # Load labels and handle mixed types
        labels = pd.read_csv('Active_Wiretap_labels.csv', 
                            names=['label'], 
                            header=None,
                            low_memory=False)
        
        # Ensure datasets have the same number of rows
        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        
        # Convert labels to numeric
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')
        
        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return

    # Preprocess the data
    print("\nPreprocessing data...")
    n_features = df.shape[1]
    column_names = [f"feature_{i}" for i in range(n_features)]
    df.columns = column_names
    df['label'] = labels['label']

    # Optimize memory usage
    print("Optimizing memory usage...")
    df = optimize_dtypes(df)

    # Clean data
    print("Cleaning data...")
    df = df.dropna()  # Drop rows with missing values
    df = df[df['label'].notna()]  # Drop rows where label is NaN

    print(f"Shape after cleaning: {df.shape}")

    # Split features and labels
    X = cp.array(df.drop('label', axis=1).values, dtype=cp.float32)
    y = cp.array(df['label'].values, dtype=cp.int32) 

    # Split the data
    print("\nSplitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X.get(), y.get(), test_size=0.3, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Feature selection
    print("Performing feature selection...")
    selector = SelectKBest(score_func=f_classif, k=min(30, X_train.shape[1]))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Train model
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train_selected, y_train)
    print("Model training completed!")

    # Make predictions
    print("Making predictions...")
    y_pred = model.predict(X_test_selected)

    # Model evaluation
    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Additional information
    print("\nProcess completed!")
    print("\nDataset Information:")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{pd.Series(y.get()).value_counts()}")

if __name__ == "__main__":
    mainCuPY()


Loading dataset...
Dataset shape: (2278689, 115)
Labels shape: (2278689, 1)
Label values found: [nan  0.  1.]

Preprocessing data...
Optimizing memory usage...
Cleaning data...
Shape after cleaning: (2278688, 116)

Splitting data into train and test sets...
Training set shape: (1595081, 115), Testing set shape: (683607, 115)
Performing feature selection...


C:\Users\antho\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:97: RuntimeWarning: overflow encountered in square
  square_of_sums_alldata = sum(sums_args) ** 2
C:\Users\antho\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:98: RuntimeWarning: overflow encountered in square
  square_of_sums_args = [s**2 for s in sums_args]
C:\Users\antho\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:103: RuntimeWarning: invalid value encountered in subtract
  ssbn -= square_of_sums_alldata / float(n_samples)
C:\Users\antho\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:104: RuntimeWarning: invalid value encountered in subtract
  sswn = sstot - ssbn



Training the MLP model...
Model training completed!
Making predictions...

Model Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97    406644
           1       0.95      0.97      0.96    276963

    accuracy                           0.97    683607
   macro avg       0.96      0.97      0.96    683607
weighted avg       0.97      0.97      0.97    683607


Confusion Matrix:
[[392342  14302]
 [  9535 267428]]

Process completed!

Dataset Information:
Total samples: 2278688
Number of features: 115
Number of selected features: 30
Label distribution:
0    1355473
1     923215
Name: count, dtype: int64


In [4]:
patch_sklearn() # Apply Intel optimizations

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

def mainIntelex():
    print("Loading dataset...")
    try:
        # Load main dataset
        df = pd.read_csv('Active_Wiretap_dataset.csv', 
                         header=None,
                         low_memory=False)
        
        # Load labels and handle mixed types
        labels = pd.read_csv('Active_Wiretap_labels.csv', 
                            names=['label'], 
                            header=None,
                            low_memory=False)
        
        # Ensure datasets have the same number of rows
        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        
        # Convert labels to numeric
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')
        
        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return

    # Step 2: Preprocess the data
    print("\nPreprocessing data...")
    n_features = df.shape[1]
    column_names = [f"feature_{i}" for i in range(n_features)]
    df.columns = column_names
    df['label'] = labels['label']

    # Step 3: Optimize memory usage
    print("Optimizing memory usage...")
    df = optimize_dtypes(df)

    # Step 4: Clean data
    print("Cleaning data...")
    # Remove any rows with missing values
    df = df.dropna()
    # Remove any rows where the label is NaN
    df = df[df['label'].notna()]

    print(f"Shape after cleaning: {df.shape}")

    # Step 5: Split features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    # Step 6: Split the data
    print("\nSplitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Step 7: Feature selection
    print("Performing feature selection...")
    selector = SelectKBest(score_func=f_classif, k=min(30, X.shape[1]))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Step 8: Train model
    print("\nTraining the MLP model...")
    model = MLPClassifier(
        hidden_layer_sizes=(50, 30),  
        max_iter=1200,               
        learning_rate='adaptive',            
        early_stopping=True,        
        random_state=42              
    )
    model.fit(X_train_selected, y_train)
    print("Model training completed!")

    # Step 9: Predictions
    print("Making predictions...")
    y_pred = model.predict(X_test_selected)

    # Step 10: Evaluation
    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Additional information
    print("\nProcess completed!")

    print("\nDataset Information:")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{y.value_counts()}")

if __name__ == "__main__":
    mainIntelex()

Loading dataset...
Dataset shape: (2278689, 115)
Labels shape: (2278689, 1)
Label values found: [nan  0.  1.]

Preprocessing data...
Optimizing memory usage...
Cleaning data...
Shape after cleaning: (2278688, 116)

Splitting data into train and test sets...
Performing feature selection...

Training the MLP model...
Model training completed!
Making predictions...

Model Evaluation:

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    406644
         1.0       0.94      0.98      0.96    276963

    accuracy                           0.97    683607
   macro avg       0.96      0.97      0.97    683607
weighted avg       0.97      0.97      0.97    683607


Confusion Matrix:
[[389456  17188]
 [  4378 272585]]

Process completed!

Dataset Information:
Total samples: 2278688
Number of features: 115
Number of selected features: 30
Label distribution:
label
0.0    1355473
1.0     923215
Name: count, dtype: int64
